<b> <font size="5">TracHack 21.2 Sample Notebook </font> </b>

*Version: 0.1*

*Author: Nandini Nuthalapati - nnuthalapati@tracfone.com* 

*Contact: If you have any questions, contact TracHackAdmin@tracfone.com*

This is a quick walk through how to get started with the TracHack ( http://trachack.com ) 21.2 dataset and Python. It is not exhaustive, but meant to give you starting point with loading in the data frame, creating samples, writing datasets out.

There are references and links to resources towards the end that you would find handy.

This was developed on AWS EMR instance with 
 * 1x Master - m4.xlarge - 4 vCore, 16 GiB memory, EBS only storage - EBS Storage:100 GiB

# Reading in full Dev Data
You should have access to the model development datasets. These are csv files named upgrades.csv, customer_info.csv, suspensions.csv, redemptions.csv, reactivations.csv, deactivations.csv, phone_info.csv, lrp_enrollment.csv, lrp_points.csv and network_usage_domestic.csv

<b>upgrades.csv</b> - base dataset that has line_id, date_observed and upgrade columns.

<b>customer_info.csv</b> - customer info has carrier, plan and activation information for each line_id. One line_id in upgrades data has one and only record in customer info, hence you can do inner join on both. 

<b>customer_info.csv</b> - customer info has carrier, plan and activation information for each line_id. One line_id in upgrades data has one and only record in customer info, hence inner join would be the right choice for both.

<b>phone_info.csv</b> - phone info has all the device information for each line_id. One line_id in upgrades data has one and only record in phone info, hence inner join would be the right choice for both.

<b>redemptions.csv</b> - redemptions has all the plan redemption details for each line_id. One line_id in upgrades data has more than one record in redemptions, hence inner join would be the right choice for both.

<b>deactivations.csv</b> - deactivations has the deactivation details for each line_id. One line_id in upgrades data has more than one record in deactivations, hence inner join would be the right choice for both.

<b>suspensions.csv</b> - suspension is when a customer is more than 15 days past due. One line_id in upgrades data may not have a record in suspensions, hence left join would be the right choice for both.

<b>network_usage_domestic.csv</b> - domestic network usage has the network usage details for each line_id. One line_id in upgrades data has more than one record in domestic network usage, hence inner join would be the right choice for both.

<b>lrp_points.csv</b> - lrp points has the loyalty reward details for each line_id. One line_id in upgrades data may not have a record in lrp points, hence left join would be the right choice for both.

<b>lrp_enrollment.csv</b> - lrp enrollment has the loyalty reward enrollment details for each line_id. One line_id in upgrades data may not have a record in lrp enrollment, hence left join would be the right choice for both.

In [32]:
# installing 1.0.3 because this version of pandas supports write to s3
!pip install pandas==1.0.3

  Using cached pandas-1.0.3-cp37-cp37m-manylinux1_x86_64.whl (10.0 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.3
    Uninstalling pandas-1.1.3:
      Successfully uninstalled pandas-1.1.3
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


## Upgrades

In [30]:
# This path will be active after the launch of the hackathon
teamname = 'trachack-a-groups-admin-py-tracfone'
data_folder='s3://tf-trachack-data/212/'
# change root_folder to your team's root folder
# s3://tf-trachack-notebooks/<this should be replaced by team name as provided in EMAIL>/jupyter/jovyan/
root_folder='s3://tf-trachack-notebooks/'+teamname+'/jupyter/jovyan/'

In [61]:
import pandas as pd
upgrades=pd.read_csv(data_folder+"data/dev/upgrades.csv")
upgrades.head(2)

,line_id,date_observed,upgrade
0,f0b99d3b-32f7-4464-8fa8-87f27c9810a8,2021-03-14,yes
1,a2ec0baf-1720-4b1b-bfa3-db00ac8fcb00,2021-03-10,yes


In [62]:
len(upgrades)

55868

The goal of the challenge is to predict upgrade. These examples have labels from historical data.

In [63]:
upgrades['upgrade'].value_counts()

no     40694
yes    15174
Name: upgrade, dtype: int64

## Customer Info

In [64]:
customer_info=pd.read_csv(data_folder+"data/dev/customer_info.csv")
customer_info.head(2)

,line_id,carrier,first_activation_date,plan_name,plan_subtype,redemption_date
0,e1b8b731-f3c6-4a8a-b09c-f3556888c868,carrier 1,2017-12-12,NaN,NaN,NaN
1,51c1e947-f8a4-4a9c-94f8-45f52198f329,carrier 1,2021-01-18,plan 2,[NULL],2021-02-25


In [65]:
len(customer_info)

55868

## Phone Info

In [66]:
phone_info=pd.read_csv(data_folder+"data/dev/phone_info.csv")
phone_info.head(2)

,line_id,cpu_cores,expandable_storage,gsma_device_type,gsma_model_name,gsma_operating_system,internal_storage_capacity,lte,lte_advanced,lte_category,manufacturer,os_family,os_name,os_vendor,os_version,sim_size,total_ram,touch_screen,wi_fi,year_released
0,8803884c-38df-4243-a522-891c2117790c,4,0.0,Smartphone,iPhone 7 (A1779),iOS,32/128/256,1.0,1.0,9.0,Apple,iOS,iOS,Apple,10,Nano,2048,1.0,1.0,2016.0
1,75b2d7a7-c4f8-4cf2-af32-ab25b5dffacc,8,1.0,Smartphone,SM-N975U,Android,256,1.0,1.0,20.0,Samsung,Android,Android,Google,9,Nano,8192,1.0,1.0,2019.0


In [67]:
len(phone_info)

55868

## Redemptions

In [68]:
redemptions=pd.read_csv(data_folder+"data/dev/redemptions.csv")
redemptions.head(2)

,line_id,channel,gross_revenue,redemption_date,redemption_type,revenue_type
0,003a475f-a35c-4d9d-b72d-178dede369e3,APP,35.0,2020-08-31,MPPAID,MPPAID
1,a753a5d8-3260-41bc-aba0-84dde1c7c4c0,WEB,50.0,2021-01-28,MPPAID,MPPAID


In [69]:
len(redemptions)

641493

## Deactivations

In [70]:
deactivations=pd.read_csv(data_folder+"data/dev/deactivations.csv")
deactivations.head(2)

,line_id,deactivation_date,deactivation_reason
0,bbc9e615-39c0-4c2f-93b1-67a1f50cacda,2020-06-11,PASTDUE
1,61c2a41e-f827-4c0c-a5cd-97396c5ed8b7,2019-06-02,PASTDUE


In [71]:
len(deactivations)

256754

## Reactivations

In [72]:
reactivations=pd.read_csv(data_folder+"data/dev/reactivations.csv")
reactivations.head(2)

,line_id,reactivation_channel,reactivation_date
0,f72d37a7-50a0-4ed0-95e6-5aaa47c8691f,WEB,2020-08-01
1,b71f5bed-3e65-48b2-ae3f-d8b1a427b29f,WEB,2020-01-23


In [73]:
len(reactivations)

248597

## Suspensions

In [74]:
suspensions=pd.read_csv(data_folder+"data/dev/suspensions.csv")
suspensions.head(2)

,line_id,suspension_start_date,suspension_end_date
0,13f3ff31-613f-4e8f-85c5-f22a450a77cc,2015-11-16,2015-11-16
1,00d88e15-9353-4641-a71c-ff46f79c178d,2019-03-30,2019-03-30


In [75]:
len(suspensions)

535849

## Network Usage Domestic

In [76]:
network_usage_domestic=pd.read_csv(data_folder+"data/dev/network_usage_domestic.csv")
network_usage_domestic.head(2)

,line_id,date,hotspot_kb,kb_5g,mms_in,mms_out,sms_in,sms_out,total_kb,voice_count_in,voice_count_total,voice_min_in,voice_min_out
0,24daa57d-c261-444c-afd4-c107a53d638e,2021-02-04,0.000000,0.0,0,0,10,7,7.533189e+05,5,8,7,4
1,8ba9b32c-3df3-4145-81ac-f334763a6f43,2021-02-13,166455.194336,0.0,0,0,6,9,2.750695e+06,0,2,0,3


In [77]:
len(network_usage_domestic)

6381966

## LRP Points

In [78]:
lrp_points=pd.read_csv(data_folder+"data/dev/lrp_points.csv")
lrp_points.head(2)

,line_id,quantity,status,total_quantity,update_date
0,77f81f11-ed62-438d-9c04-02e3bcc0d756,2180.0,ENROLLED,2180.0,2021-01-02
1,d664604d-8b68-4302-b599-0165df0f27ad,1210.0,ENROLLED,1210.0,2021-01-03


In [79]:
len(lrp_points)

8860

## LRP Enrollment

In [80]:
lrp_enrollment=pd.read_csv(data_folder+"data/dev/lrp_enrollment.csv")
lrp_enrollment.head(2)

,line_id,lrp_enrolled,lrp_enrollment_date
0,487ddc53-0fe4-429e-950d-8f3de2231883,Y,2019-12-02
1,081267e8-6fd7-467e-96e2-758cd8597800,Y,2017-04-30


In [81]:
len(lrp_enrollment)

17634

# Joining Tables
Below is the example of how to join upgrades, customer info and network usage data. Below code selects only a subset of columns for demonstration.

In [82]:
upgrades_custinfo=pd.merge(upgrades,customer_info.loc[:,['line_id','carrier','plan_name']],on='line_id',how='inner')
upgrades_custinfo_network=pd.merge(upgrades_custinfo,network_usage_domestic.loc[:,['line_id','total_kb']],on='line_id',how='inner')
print(len(upgrades_custinfo_network))
print(len(upgrades_custinfo_network.columns))

6381966
6


The following writes out the sample to a single file. This can be useful for modeling on subsets of data.

In [83]:
upgrades_custinfo_network.to_csv(root_folder+"sample-notebooks/dev-sample.csv",header=True,index=None)

In [84]:
upgrades_eval=pd.read_csv(data_folder+"data/eval/upgrades.csv")
customer_info_eval=pd.read_csv(data_folder+"data/eval/customer_info.csv")
network_usage_domestic_eval=pd.read_csv(data_folder+"data/eval/network_usage_domestic.csv")

In [88]:
upgrades_custinfo=pd.merge(upgrades_eval.loc[:,['line_id']],customer_info_eval.loc[:,['line_id','carrier','plan_name']],on='line_id',how='inner')
upgrades_custinfo_network=pd.merge(upgrades_custinfo,network_usage_domestic_eval.loc[:,['line_id','total_kb']],on='line_id',how='inner')
print(len(upgrades_custinfo_network))
print(len(upgrades_custinfo_network.columns))

4253608
4


In [86]:
upgrades_custinfo_network.to_csv(root_folder+"sample-notebooks/eval-sample.csv",header=True,index=None)